In [ ]:
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import utils  # Custom utility functions for training and evaluation
from PIL import Image

In [ ]:
# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    # Add more transformations as needed (e.g., normalization, resizing)
])

In [ ]:
import torch
from torch.utils.data import Dataset


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        target = self.targets[index]

        if self.transform:
            sample = self.transform(sample)

        return sample, target


In [ ]:
# Load custom dataset
train_dataset = CustomDataset(data='D:\AI Project Guidelines\project 2\data2\images\train', targets='D:\AI Project Guidelines\project 2\data2\labels\train', transform=transform)
test_dataset = CustomDataset(data='D:\AI Project Guidelines\project 2\data2\images\test', targets='D:\AI Project Guidelines\project 2\data2\labels\test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [ ]:
# Load pre-trained Faster R-CNN model
model = fasterrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# Modify the model's final layer to match the number of classes in your dataset
num_classes = 2  # Number of classes (bike, person)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
# Define device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

In [ ]:
# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
# Define learning rate scheduler (optional)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)